In [27]:
from dotenv import load_dotenv

load_dotenv()

True

In [28]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

In [29]:
from langchain.tools import tool

@tool
def sql_query(query: str) -> str:
    """
    Obtain information from the database using SQL queries
    """
    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

sql_query.invoke("SELECT * FROM Artist LIMIT 10")

"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [30]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=[sql_query]
)

In [31]:
from langchain.messages import HumanMessage

user_query = HumanMessage(
    content="Who is the most popular artist beginning with 'S' in this database?"
)

response = agent.invoke(
    {"messages": [user_query]}
)

In [32]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content="Who is the most popular artist beginning with 'S' in this database?", additional_kwargs={}, response_metadata={}, id='4b9585ae-baa5-4a93-a639-ff7d80af5aa4'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 616, 'prompt_tokens': 142, 'total_tokens': 758, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CtdEL8yUVO7ej8glupnLX2BM6YK8Z', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7fc0-7259-7133-a623-d108b6921d44-0', tool_calls=[{'name': 'sql_query', 'args': {'query': "SELECT name FROM artists WHERE name LIKE 'S%' ORDER BY popularity DESC LIMIT 1;"}, 'id': 'call_

In [33]:
print(response["messages"][-3].tool_calls[0]["args"]["query"])

SELECT a.Name, SUM(il.Quantity * il.UnitPrice) AS Revenue
FROM Artist a
JOIN Track t ON t.AlbumId IN (SELECT AlbumId FROM Album WHERE Album.ArtistId = a.ArtistId)
JOIN InvoiceLine il ON il.TrackId = t.TrackId
WHERE a.Name LIKE 'S%'
GROUP BY a.Name
ORDER BY Revenue DESC
LIMIT 1;


https://smith.langchain.com/public/f1cc341e-d78b-4e0d-bcd2-798cd3271435/r

## Improved method by providing DB Schema as Context to Agent

In [34]:
from dataclasses import dataclass
from langchain_community.utilities import SQLDatabase
from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent
from langchain.messages import HumanMessage, SystemMessage

DB_URI = "sqlite:///resources/Chinook.db"

@dataclass
class SQLContext:
    db: SQLDatabase
    schema: str # one-time snapshot


def build_sql_context(db_uri: str) -> SQLContext:
    db = SQLDatabase.from_uri(db_uri)
    schema = db.get_table_info() # one-time schema capture
    return SQLContext(db=db, schema=schema)

ctx = build_sql_context(DB_URI)

In [35]:
pprint(ctx)

SQLContext(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x11554fbc0>,
           schema='\n'
                  'CREATE TABLE "Album" (\n'
                  '\t"AlbumId" INTEGER NOT NULL, \n'
                  '\t"Title" NVARCHAR(160) NOT NULL, \n'
                  '\t"ArtistId" INTEGER NOT NULL, \n'
                  '\tPRIMARY KEY ("AlbumId"), \n'
                  '\tFOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")\n'
                  ')\n'
                  '\n'
                  '/*\n'
                  '3 rows from Album table:\n'
                  'AlbumId\tTitle\tArtistId\n'
                  '1\tFor Those About To Rock We Salute You\t1\n'
                  '2\tBalls to the Wall\t2\n'
                  '3\tRestless and Wild\t2\n'
                  '*/\n'
                  '\n'
                  '\n'
                  'CREATE TABLE "Artist" (\n'
                  '\t"ArtistId" INTEGER NOT NULL, \n'
                  '\t"Name" NVARCHAR(120), \n'
 

In [36]:
@tool
def get_db_schema(runtime: ToolRuntime) -> str:
    """
    Get the database schema (one-time snapshot)
    """
    return runtime.context.schema


@tool
def sql_query(runtime: ToolRuntime, query: str) -> str:
    """
    Run a SQL query against the database
    """
    try:
        return runtime.context.db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [37]:
system_prompt = SystemMessage(
    content=(
        "You are a SQL assistant. Always called `get_db_schema` before writing SQL, "
        "and use only tables/columns that appear there."
    )
)


agent = create_agent(
    model="gpt-5-nano",
    tools=[get_db_schema, sql_query],
    context_schema=SQLContext,
    system_prompt=system_prompt
)

In [38]:
user_query = HumanMessage(
    content="Who is the most popular artist beginning with 'S'?"
)

response = agent.invoke(
    {"messages": [user_query]},
    context=ctx
)

/Users/johnmachado/Desktop/LangChain 1.0 Foundations/env/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=SQLContext(db=<langchain_...619\t3990994\t0.99\n*/'), input_type=SQLContext])
  return self.__pydantic_serializer__.to_python(
/Users/johnmachado/Desktop/LangChain 1.0 Foundations/env/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=SQLContext(db=<langchain_...619\t3990994\t0.99\n*/'), input_type=SQLContext])
  return self.__pydantic_serializer__.to_python(


In [39]:
pprint(response)

{'messages': [HumanMessage(content="Who is the most popular artist beginning with 'S'?", additional_kwargs={}, response_metadata={}, id='16443494-c044-4e06-a8e3-ed45e07f736e'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 276, 'prompt_tokens': 185, 'total_tokens': 461, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CtdEtXiQstRk5oBnzY0YSxJfBdMv0', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7fc0-f388-7402-96a4-4c376956123c-0', tool_calls=[{'name': 'get_db_schema', 'args': {}, 'id': 'call_Ss5oCzYKE2dVQtOAhIydvdpg', 'type': 'tool_call'}], usage_metadata={'input_tokens': 185, 'output_tokens':

https://smith.langchain.com/public/478a4f27-5ae3-4c1e-b17f-90ea5892dd4a/r

Before Providing Schema to LLM:

- 11 SQL queries
- 32 seconds to get final answer


After Providing Schema to LLM:

- 1 SQL query
- 16 seconds to get final answer
